In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
from datetime import timedelta
from typing import List, Dict
import random

import pandas as pd
pd.set_option('display.max_rows', 200)
import numpy as np

from tqdm import tqdm

import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp

from data_handlers.runtimes import *

In [28]:
experiment_path = pathlib.Path(f"/home/sterz/maci-docker-compose/maci_data/binary_files")
experiment_ids = [1, 2, 3, 4, 5]

In [29]:
event_frame = parse_bundle_events(experiment_path, experiment_ids)

Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/1/101
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/1/84
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/1/60
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/1/17
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/1/133
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/1/144
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/1/139
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/1/58
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/1/25
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/1/52
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/1/118
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/1/112
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/1/73
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/1/79
Parsing /home/sterz/maci-docker-compose/ma

/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/

Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/181
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/165
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/288
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/385
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/211
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/266
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/361
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/316
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/282


/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/

Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/359
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/254
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/223
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/324
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/353
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/229
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/176
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/198
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/192
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/208
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/372
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/305
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/291
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/2/396
Parsing /home/sterz/maci-docker-co

...

/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/

Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/644
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/644
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/639
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/639
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/591
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/591
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/575
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/575
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/698
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/698
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/676
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/676
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/601
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/601
Parsing /home/sterz/maci-docker-co

/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/

Parsing done


/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/

In [30]:
creation_instances = event_frame[event_frame.Event == "creation"].groupby(["Simulation ID"])
delivery_instances = event_frame[event_frame.Event == "delivery"]

In [33]:
delivered_list = []
runtime_list = []
for sim_instance_id, creations in tqdm(creation_instances):
    deliveries_sid = delivery_instances[delivery_instances["Simulation ID"] == sim_instance_id]
    
    for _, row in creations.iterrows():
        # Compute deliverys
        bundle = row.Bundle
        deliveries = deliveries_sid[deliveries_sid.Bundle == bundle]

        was_delivered = not deliveries.empty

        row_dict = {
            "Simulation ID": sim_instance_id,
            "Bundle": bundle,
            "Delivered": was_delivered,
            "CLA": row.CLA,
            "Payload Size": row["Payload Size"],
            "Bundles per Second": row["Bundles per Second"],
            "Software": row.Software,
            "# Nodes": row["# Nodes"],
            "# Payloads": row["# Payloads"],
        }
        delivered_list.append(row_dict)        
        
        # Compute runtimes
        if was_delivered:
            delivery_time = deliveries["Timestamp"].min()
            creation_time = row["Timestamp"]
            runtime = delivery_time - creation_time
            
            runtime_dict = {
                "Simulation ID": sim_instance_id,
                "Bundle": bundle,
                "Runtime": (runtime.to_pytimedelta() / timedelta(milliseconds=1)) / 1000,
                "CLA": row.CLA,
                "Software": row.Software,
                "# Nodes": row["# Nodes"],
                "Payload Size": row["Payload Size"],
                "Bundles per Second": row["Bundles per Second"],
                "# Payloads": row["# Payloads"],
            }
            runtime_list.append(runtime_dict)
        
delivered_df = pd.DataFrame(delivered_list)
runtime_df = pd.DataFrame(runtime_list)

100%|██████████| 729/729 [01:00<00:00, 12.09it/s]


In [36]:
fig = px.box(
    runtime_df,
    x="Software", 
    y="Runtime",
    color="CLA",
    facet_col="# Nodes",
    facet_row="# Payloads"
)

fig.show()

In [37]:
ratio_df = pd.DataFrame()
for sim_instance_id, bundles in delivered_df.groupby(["Simulation ID"]):
    total_bundles = len(bundles.index)
    delivered_bundles = len(bundles[bundles.Delivered == True].index)
    
    cla = bundles["CLA"].values[0]
    software = bundles["Software"].values[0]
    nodes = bundles["# Nodes"].values[0]
    payload_size = bundles["Payload Size"].values[0]
    bundles_per_second = bundles["Bundles per Second"].values[0]
    payloads = bundles["# Payloads"].values[0]
    
    row_dict = {
        "Simulation ID": sim_instance_id,
        "Delivered (%)": (float(delivered_bundles) / float(total_bundles)) * 100,
        "CLA": cla,
        "Software": software,
        "# Nodes": nodes,
        "Payload Size": payload_size,
        "Bundles per Second": bundles_per_second,
        "# Payloads": payloads,
    }
    
    runtime_dict = {
    }
    ratio_df = ratio_df.append([row_dict], ignore_index=True)

/tmp/ipykernel_3435800/2612813690.py:2: FutureWarning:

In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.

/tmp/ipykernel_3435800/2612813690.py:26: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_3435800/2612813690.py:26: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_3435800/2612813690.py:26: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_3435800/2612813690.py:26: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_3435800/2612813690.p

In [39]:
fig = px.histogram(
    ratio_df,
    x="Software",
    y="Delivered (%)",
    color="CLA",
    facet_col="# Nodes",
    facet_row="# Payloads",
    barmode='group',
    histfunc='avg',
)

fig.show()